In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Fri Sep 16 13:45:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#pip install tensorflow-gpu

In [ ]:
# Import des librairies utiles 
import tensorflow
import keras
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
#from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Input, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
import pickle

In [ ]:
# Avant de lancer la construction du modèle CNN basique on étudie les données disponibles 
import os 
X_train_cat = os.listdir('drive/MyDrive/Interface_E2/clean_dataset/train/cat')
X_val_cat = os.listdir('drive/MyDrive/Interface_E2/clean_dataset/validation/cat')
X_test_cat = os.listdir('drive/MyDrive/Interface_E2/clean_dataset/test/cat')
print("Nous avons " + str(len(X_train_cat)) + " images d'entrainement de chats.")
print("Nous avons " + str(len(X_val_cat)) + " images de validation de chats.")
print("Nous avons " + str(len(X_test_cat)) + " images de test de chats.")

Nous avons 9987 images d'entrainement de chats.
Nous avons 1248 images de validation de chats.
Nous avons 1248 images de test de chats.


In [ ]:
X_train_dog = os.listdir('drive/MyDrive/Interface_E2/clean_dataset/train/dog')
X_val_dog = os.listdir('drive/MyDrive/Interface_E2/clean_dataset/validation/dog')
X_test_dog = os.listdir('drive/MyDrive/Interface_E2/clean_dataset/test/dog')
print("Nous avons " + str(len(X_train_dog)) + " images d'entrainement de chiens.")
print("Nous avons " + str(len(X_val_dog)) + " images de validation de chiens.")
print("Nous avons " + str(len(X_test_dog)) + " images de test de chiens.")

Nous avons 9985 images d'entrainement de chiens.
Nous avons 1248 images de validation de chiens.
Nous avons 1248 images de test de chiens.


In [ ]:
# De même que pour la régression logistique, une étape de preprocessing est nécessaire. 
# Elle permettra de standardiser les dimensions des images et de normaliser les valeurs des pixels. 
# En accord avec l'architecture VGG16, nous utiliserons des images de 224*224*3. 
# Pour cela on instancie la méthode ImageDataGenerator de Keras.preprocessing qui permet de normaliser les valeurs de pixels. 
# On utilise cette méthode car elle permettra dans un autre modèle de réaliser de la data augmentation online.
preproc = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
# A l'aide de la fonction flow_from_directory() qui nécessite que la structure des dossiers soit celle présentée dans le rapport, 
# Nous allons pouvoir lire les images directement depuis leurs dossiers respectifs.
# Cette méthode permet de fixer la taille des images avec l'argument target_size
# Elle créera des batch d'images. 
# Un batch est caractérisé par sa taille et définit le nombre d'images à traiter par le modèle 
# avant de mettre à jour les poids du modèle. Une valeur commune de batch est 32 ou 64.
# L'avantage d'utiliser flow_from_directory est qu'il infère les labels en se basant sur le nom du dossier.
train_set = preproc.flow_from_directory('drive/MyDrive/Interface_E2/clean_dataset/train/',
                                        class_mode = 'binary',
                                        batch_size = 64,
                                        target_size = (224, 224))

validation_set = preproc.flow_from_directory('drive/MyDrive/Interface_E2/clean_dataset/validation',
                                             class_mode = 'binary',
                                             batch_size = 64,
                                             target_size = (224, 224))

test_set = preproc.flow_from_directory('drive/MyDrive/Interface_E2/clean_dataset/test',
                                       class_mode = 'binary', 
                                       batch_size = 64, 
                                       target_size = (224, 224))

Found 19972 images belonging to 2 classes.
Found 2496 images belonging to 2 classes.
Found 2496 images belonging to 2 classes.


In [ ]:
# Une fois les données préparer on peut définir le modèle basique

def basic_cnn(): 
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(1, activation='sigmoid'))
  
  # On doit compiler le modèle en spécifiant le solver = algorithme utilisé pour minimiser l'erreur de prédiction
  # Les valeurs de learning rate et du momentum sont choisies en accord avec les hyperparamètre de VGG16 
  solver = SGD(learning_rate=0.001, momentum=0.9)
  model.compile(optimizer=solver, loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
# On définit la fonction plot_perf() qui enregistrera deux graphiques d'accuracy et de loss pour le
# set d'entrainement et de validation.
# On précise que history.history contient les différentes métriques de performances réalisées. 
# Il s'agit de la loss et de l'accuracy

def plot_perf(history): 
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    # On regarde l'évolution en fonction des epochs 
    epochs = range(len(acc))
    
    # On plot l'accuracy de l'entrainement et de la validation par epoch
    plt.plot(epochs, acc, color="blue", label='Train')
    plt.plot(epochs, val_acc, color="orange", label="Val")
    plt.title('Accuracy sur Train et Validation')
    #plt.ylim(yrange)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    filename1 = "Train_Val_Acc_BasicCNN"
    plt.savefig(filename1 + '_plot.png')
    plt.close()
    
    plt.figure()
    plt.plot(epochs, loss, color="blue", label='Train')
    plt.plot(epochs, val_loss, color="orange", label="Val" )
    plt.title('Loss sur Train et Validation')
    #plt.ylim(yrange)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    filename2 = "Train_Val_Loss_BasicCNN"
    plt.savefig(filename2 + '_plot.png')
    plt.close()

In [ ]:
# On créer une fonction qui regroupe les précédente et qui permet de lancer toutes les opérations.
def launch_learning(): 
    
    # On définit le modèle
    model = basic_cnn()
    
    # On fit le modèle sur les données d'apprentissages et on donne les données de validation
    # history contient alors les valeurs de loss et d'accuracy
    # Le paramètre verbose permet d'afficher le temps passé par epoch, et l'accuracy et la loss
    history = model.fit(train_set, 
                       validation_data=validation_set, 
                       epochs=25, 
                       verbose=1)
    
    # On évalue le modèle sur le jeux de données test
    # Par défaut le paramètre setps est None. L'évaluation prendra en compte tous les batch avant de s'arrêter.
    evaluation = model.evaluate(test_set,
                                batch_size=64,
                                verbose=1)
    print(model.metrics_names, evaluation)
    filename="Basic_CNN.sav"
    pickle.dump(model, open(filename, 'wb'))

    plot_perf(history)
    

In [ ]:
launch_learning()

Epoch 1/25
313/313 [==============================] - 84s 262ms/step - loss: 0.7065 - accuracy: 0.5307 - val_loss: 0.6831 - val_accuracy: 0.5244
Epoch 2/25
313/313 [==============================] - 78s 251ms/step - loss: 0.6736 - accuracy: 0.5844 - val_loss: 0.6645 - val_accuracy: 0.6030
Epoch 3/25
313/313 [==============================] - 77s 245ms/step - loss: 0.6577 - accuracy: 0.6148 - val_loss: 0.6629 - val_accuracy: 0.6094
Epoch 4/25
313/313 [==============================] - 77s 246ms/step - loss: 0.6593 - accuracy: 0.6120 - val_loss: 0.6664 - val_accuracy: 0.6078
Epoch 5/25
313/313 [==============================] - 77s 247ms/step - loss: 0.6549 - accuracy: 0.6211 - val_loss: 0.6552 - val_accuracy: 0.6298
Epoch 6/25
313/313 [==============================] - 77s 247ms/step - loss: 0.6482 - accuracy: 0.6294 - val_loss: 0.6484 - val_accuracy: 0.6258
Epoch 7/25
313/313 [==============================] - 78s 250ms/step - loss: 0.6414 - accuracy: 0.6395 - val_loss: 0.6618 - val_ac